In [1]:
import pandas as pd
import numpy as np

def read_DB(db_path):
    db_sh = pd.ExcelFile(db_path)
    sheets = db_sh.sheet_names
    db = pd.read_excel(db_path, sheet_name= sheets, index_col= 0)
    # add 
    for col in sheets:
        if col == 'Types':
            db[col]['descOrigin'] = db[col]['Type'].astype(str) + ', ' + db[col]['Origin'].astype(str)
        elif col == 'References': 
            db[col]['authorYear'] = db[col]['Author'].astype(str) + ', ' + db[col]['Publication Year'].astype(str)
        elif col == 'Country':
            db[col]['descOrigin'] = db[col]['Country'].astype(str) + ', ' + db[col]['City'].astype(str)  
        elif col == 'Region':
            pass
        else:
            db[col]['descOrigin'] = db[col]['Description'].astype(str) + ', ' + db[col]['Origin'].astype(str)
    return db


In [ ]:
db_path = r'C:\GitHub\suews_prepare_database\input\database.xlsx'
db_dict = read_DB(db_path)

# key is typology code 
# value is volumetric fraction for specific grid
# in this GridID. 0.45 of buildings in grid is typology 128 and .55 is 1210

frac_dict = { 
    128: 0.45, 
    1210: 0.55}

# weights for np.average()
weights = list(frac_dict.values())

for key in list(frac_dict.keys()):
    building_sel = db_dict['Types'].loc[key, 'Buildings']               # retrieve selected typology code
    SS_surf = db_dict['NonVeg'].loc[building_sel, 'Spartacus Surface']  # retrieve Spartacus code for typology
    SS_surf_sel = db_dict['Spartacus Surface'].loc[[SS_surf]]           # retrieve attributes for selected Spartacus Code
    for material in SS_surf_sel.loc[:, SS_surf_sel.columns.str.endswith('Material')]:
        material_code = SS_surf_sel.loc[:,material].item()
        if material_code != -9999:
            SS_material = db_dict['Spartacus Material'].loc[material_code]
    
# averaged_values = np.average(values_to_blend, weights) #
